<a href="https://colab.research.google.com/github/euphoria96/KB_Smishing_dacon/blob/master/Baseline_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting
### Mounting google drive

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

### Download Mecab
Dacon에서 공개한(?) Window Mecab version

In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab

/content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab


In [0]:
! bash install_mecab-ko_on_colab190912.sh

### Import packages

In [0]:
# data preprocessing
import pandas as pd
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm # for checking time
import warnings
warnings.filterwarnings(action='ignore') 

### Load Data

In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/

/content/gdrive/My Drive/Colab Notebooks/smishing


In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("public_test.csv")
submission=pd.read_csv("submission_제출양식.csv")
train.shape, test.shape, submission.shape

((295945, 4), (8105, 3), (8105, 2))

In [0]:
train.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


총 train data의 행은 약 29만개, test data의 행은 약 8천개로 되어있다.

## Data Preparation
### Sampling
1. Under Sampling : smishing 수에 맞추기
2. Over Sampling : non-smishing 수에 맞추기
3. mixed Sampling : 

In [0]:
Counter(train['smishing'])  # 0.0631975535994864

Counter({0: 277242, 1: 18703})

In [0]:
RANDOM_SEED = 1234

In [0]:
## for train data
# index
train_nsm_idx=list(train[train['smishing']==0].index)
train_sm_idx=list(train[train['smishing']==1].index)
# 3. Mixed Sampling
random.seed(RANDOM_SEED)
train_nsm_idx = random.choices(train_nsm_idx, k=18703*2)
random.seed(RANDOM_SEED)
train_sm_idx = random.choices(train_sm_idx, k=18703*2)

In [0]:
class a:
    foo = "bar"

a1 = a()
a2 = a()
b = [a1,a2]

random.shuffle(b)

for i in b:
    print(i.foo)

bar
bar


In [0]:
train_idx = train_nsm_idx + train_sm_idx
print(random.shuffle(train_idx))


None


In [0]:
train_idx = train_idx[smp_idx]
train_idx

TypeError: ignored

In [0]:
# Merge
train_idx = train_nsm_idx + train_sm_idx
train_idx = train_idx[random.sample(train_idx)]
train_X = pd.DataFrame(train['text'], columns=['text'])[train_idx, :].reset_index(drop=True)
train_y = pd.DataFrame(train['smishing'], columns=['smishing'])[train_idx, :].reset_index(drop=True)

TypeError: ignored

In [0]:
## for test data
test['smishing'] = 2 # train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_X = pd.DataFrame(test['text'])
test_y = pd.DataFrame(test['smishing'])
train_X.shape, train_y.shape, test_X.shape, test_y.shape

### Tokenizing

In [0]:
from konlpy.tag import Mecab

tokenizer = Mecab() # setting tokenizer using Mecab()

In [0]:
train_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( train_X['text'], train_y['smishing'] ) )  ] # Mecab를 활용하여 text를 토큰화 시킴
test_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( test_X['text'], test_y['smishing'] ) )  ]

37406it [00:45, 815.02it/s] 
8105it [00:01, 6110.58it/s]


In [0]:
train_doc[0][0][:10], train_doc[0][1]

([('(', 'SSO'),
  ('광고', 'NNG'),
  (')', 'SSC'),
  ('XXXBaXXX', 'SL'),
  ('고객', 'NNG'),
  ('님', 'XSN'),
  ('들', 'XSN'),
  ('뒤', 'NNG'),
  ('엔', 'JKB+JX'),
  ('XXX', 'SL')],
 1)

In [0]:
stopwords = ['XXX', 'XXXXXX', '00', 
             '.', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', '..', '.(', '%',
             '을', '를', '이', '가', '은', '는', ] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

필요없는 단어들을 제거하면서 동시에 모형에 사용하기 위한, X_train,test, Y_train,test를 생성한다

In [0]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y)) # 2 gram
    
    X_train.append(" ".join(temp))  # ' '로 구분

In [0]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_test.append(" ".join(temp))

In [0]:
len(X_train), len(Y_train), len(X_test)

(37406, 37406, 8105)

In [0]:
X_train[:5]

## Modeling
#### 1. Find the best model

In [0]:
# model setting
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score
# models
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [0]:
v = CountVectorizer()
v.fit(X_train)
vec_train_count = v.transform(X_train).toarray() # 37406 17621
vec_test_count = v.transform(X_test).toarray()

In [0]:
v2 = TfidfVectorizer()
v2.fit(X_train)
vec_train_TFIDF = v2.transform(X_train).toarray()
vec_test_TFIDF = v2.transform(X_test).toarray()

In [0]:
vec_train_count.shape, vec_train_TFIDF.shape

((37406, 17621), (37406, 17621))

In [0]:
model1 = Pipeline([
    ('model', MultinomialNB()),
])
model2 = Pipeline([
    ('model', RandomForestClassifier(random_state=RANDOM_SEED)),
])
model3 = Pipeline([
    ('model', SGDClassifier(random_state=RANDOM_SEED, loss='hinge')),
])
model4 = Pipeline([
    ('model', LogisticRegression(random_state=RANDOM_SEED)),
])
'''model5 = Pipeline([
    ('model', KNeighborsClassifier()),
])
model6 = Pipeline([
    ('model', AdaBoostClassifier(random_state=RANDOM_SEED)),
])
model7 = Pipeline([
    ('model', SVC(random_state=RANDOM_SEED)), # probability=True
])'''

models = [model1, model2, model3, model4]

In [0]:
for i, model in enumerate(models):
    scores = cross_val_score(model, vec_train_count, Y_train, cv=5, scoring = 'roc_auc')
    print(("Model{0:d}: Mean score: {1:.4f}").format(i + 1, np.mean(scores))) # AUC 평균

Model1: Mean score: 0.9976
Model2: Mean score: 0.9998
Model3: Mean score: 0.9997
Model4: Mean score: 0.9998


In [0]:
for i, model in enumerate(models):
    scores = cross_val_score(model, vec_train_TFIDF, Y_train, cv=5, scoring = 'roc_auc')
    print(("Model{0:d}: Mean score: {1:.5f}").format(i + 1, np.mean(scores)))

Model1: Mean score: 0.99965
Model2: Mean score: 0.99970
Model3: Mean score: 0.99998
Model4: Mean score: 0.99995


#### 2. Fit the best one

In [0]:
clf = LogisticRegression(random_state=RANDOM_SEED)
clf.fit(vec_train_TFIDF, Y_train)
y_pred = clf.predict_proba(vec_test_TFIDF)
y_pred_ = [ i[1]  for i in y_pred]
y_pred_[:5]

[0.0071157578204622965,
 0.01295931081927131,
 0.0016846510050238484,
 0.005598946455309039,
 0.022293780455301476]

### Submission
test data에서 각 text당 스미싱 확률을 구할 수 있으니 이를 제출양식의 smishing 변수에 넣고 csv 파일로 내보낸다

In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/submission

/content/gdrive/My Drive/Colab Notebooks/smishing/submission


In [0]:
submission['smishing'] = y_pred_

submission.to_csv("baseline_v3_logit_1234_v2.csv",index=False)